This notebook is used to create the index arrays for our edges after spatial reduction to fit our model in stan in R.

In [89]:
import xarray as xr
import numpy as np

In [90]:
tp_quantile = 0.0013141632
ws10_quantile = 11.48831558227539

Let us analyse hurricane Michael for now.

In [91]:
speeds = xr.open_dataset("../data_grib/ABBY_1968_07.grib", engine='cfgrib', backend_kwargs={'filter_by_keys': {'paramId': [165, 166]}})
rainfall = xr.open_dataset("../data_grib/ABBY_1968_07.grib", engine='cfgrib', backend_kwargs={'filter_by_keys': {'paramId': [228]}})

speeds = speeds.to_dataframe()
rainfall = rainfall.to_dataframe()

u_speed = speeds.u10
v_speed = speeds.v10
ws10 = np.sqrt(u_speed**2 + v_speed**2)

speeds["ws10"] = ws10

mask_ws10 = speeds['ws10'] > ws10_quantile
mask_tp = rainfall['tp'] > tp_quantile

temp = mask_ws10.groupby(["latitude", "longitude"]).sum()
mask_ws10_average = temp > 0

temp = mask_tp.groupby(["latitude", "longitude"]).sum()
mask_tp_average = temp > 0

combined_mask = mask_ws10_average & mask_tp_average

Ignoring index file '../data_grib/ABBY_1968_07.grib.5b7b6.idx' incompatible with GRIB file


Ignoring index file '../data_grib/ABBY_1968_07.grib.5b7b6.idx' incompatible with GRIB file


In [92]:
mask = combined_mask.unstack()
lat = mask.index.values
lon = mask.columns.values

In [93]:
# Initialise our adjacency matrix which would be NxN

N = len(lat)*len(lon)
adj = np.zeros((N, N))

adj

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(4141, 4141))

In [94]:
# Define a function to check if node i is kept after spatial reduction

lat_temp = lat[::-1]

def retained(mask, i):

    lat_i = i % len(lat)
    lon_i = i // len(lat)

    return mask.loc[lat_temp[lat_i], lon[lon_i]]

In [95]:
# Start filling out the matrix where Aij = 1 if i and j are neighbours
# Start from top left of the map i.e lon = 95, lat = 32 and go down

i,j = 0,0

# Fill in the upper triangular of the matrix

for i in range(0,N):

    #Check if node i is retained

    if retained(mask, i):

        for j in range(i+1,N):
            
            # Check if node j is retained
                
            if retained(mask, j):

                # Check if nodes i and j are neigbours

                lat_i, lon_i = i % len(lat), i // len(lat)
                lat_j, lon_j = j % len(lat), j // len(lat)

                if lat_i == lat_j and abs(lon_j - lon_i) == 1:

                    adj[i,j] = 1

                elif lon_i == lon_j and abs(lat_i - lat_j) == 1:

                    adj[i,j] = 1


In [96]:
# Filling in the lower triangular
adj_complete = adj + adj.T

Now I want to convert the Adjacency Matrix into two lists defining all our edges.

In [97]:
N_edges = int(adj.sum())
N_edges

306

In [98]:
node1 = np.zeros(N_edges)
node2 = np.zeros(N_edges)

In [99]:
# This is the number of neighbours each node has, we are using the upper triangular to avoid a double count
# i.e if nodes 3 and 9 are neigbours, we will only consider 3 --> 9 and not 9 --> 3.
neighbours = adj.sum(axis=1)
neighbours

array([0., 0., 0., ..., 0., 0., 0.], shape=(4141,))

In [100]:
i = 0

for k in range(len(neighbours)):

    count = int(neighbours[k])
    
    if count != 0:

        locations = np.nonzero(adj[k])

        for m in range(count):

            # node1 stores the start index of the edges
            node1[i] = k+1

            # node2 stores the end index of the edges
            node2[i] = locations[0][m] + 1

            i += 1 

In [101]:
node1

array([2299., 2339., 2339., 2340., 2379., 2379., 2380., 2380., 2381.,
       2386., 2386., 2387., 2420., 2420., 2421., 2421., 2422., 2422.,
       2423., 2423., 2424., 2424., 2425., 2425., 2426., 2426., 2427.,
       2427., 2428., 2461., 2461., 2462., 2462., 2463., 2463., 2464.,
       2464., 2465., 2465., 2466., 2466., 2467., 2467., 2468., 2468.,
       2469., 2469., 2470., 2502., 2502., 2503., 2503., 2504., 2504.,
       2505., 2505., 2506., 2506., 2507., 2507., 2508., 2508., 2509.,
       2509., 2510., 2510., 2511., 2511., 2512., 2543., 2543., 2544.,
       2544., 2545., 2545., 2546., 2546., 2547., 2547., 2548., 2548.,
       2549., 2549., 2550., 2550., 2551., 2551., 2552., 2552., 2553.,
       2553., 2554., 2584., 2584., 2585., 2585., 2586., 2586., 2587.,
       2587., 2588., 2588., 2589., 2589., 2590., 2590., 2591., 2591.,
       2592., 2592., 2593., 2593., 2594., 2594., 2595., 2595., 2596.,
       2625., 2626., 2626., 2627., 2627., 2628., 2629., 2629., 2630.,
       2630., 2631.,

In [102]:
node2

array([2340., 2340., 2380., 2381., 2380., 2420., 2381., 2421., 2422.,
       2387., 2427., 2428., 2421., 2461., 2422., 2462., 2423., 2463.,
       2424., 2464., 2425., 2465., 2426., 2466., 2427., 2467., 2428.,
       2468., 2469., 2462., 2502., 2463., 2503., 2464., 2504., 2465.,
       2505., 2466., 2506., 2467., 2507., 2468., 2508., 2469., 2509.,
       2470., 2510., 2511., 2503., 2543., 2504., 2544., 2505., 2545.,
       2506., 2546., 2507., 2547., 2508., 2548., 2509., 2549., 2510.,
       2550., 2511., 2551., 2512., 2552., 2553., 2544., 2584., 2545.,
       2585., 2546., 2586., 2547., 2587., 2548., 2588., 2549., 2589.,
       2550., 2590., 2551., 2591., 2552., 2592., 2553., 2593., 2554.,
       2594., 2595., 2585., 2625., 2586., 2626., 2587., 2627., 2588.,
       2628., 2589., 2629., 2590., 2630., 2591., 2631., 2592., 2632.,
       2593., 2633., 2594., 2634., 2595., 2635., 2596., 2636., 2637.,
       2626., 2627., 2667., 2628., 2668., 2629., 2630., 2670., 2631.,
       2671., 2632.,

These are indexes on the full dataset. I want to reduce it to just extreme locations.

In [103]:
mask = mask.iloc[::-1]
df_list = mask.to_numpy().flatten(order='F').tolist()

In [104]:
extreme_indices = [i for i, val in enumerate(df_list) if val]

extreme_to_new_idx = {extreme_indices[i] + 1: i + 1 for i in range(len(extreme_indices))}

In [105]:
node1_reduced = []
node2_reduced = []

for i in range(len(node1)):

    node1_reduced.append(extreme_to_new_idx[int(node1[i])])
    node2_reduced.append(extreme_to_new_idx[int(node2[i])])

Repeat the same but for the correlation matrix now. The neighbours of the correlation between node i and j are the correlations between i and its neigbours and correlations between j and its neighbours.

In [106]:
reversed_map = {v: k for k, v in extreme_to_new_idx.items()}

In [107]:
def find_indices(value, arr):
    
    return [i for i,x in enumerate(arr) if x == value]

find_indices(3, node1_reduced)

[3]

In [108]:
#Assume I have a flattened rho matrix, i want to made a rho_node1 and rho_node1

rho_node1 = []
rho_node2 = []

N = sum(df_list)

neighbours_full = adj_complete.sum(axis=1)

for k in range(len(node1_reduced)):

    i = node1_reduced[k]
    j = node2_reduced[k]

    count = int(neighbours_full[reversed_map[i]-1]) + int(neighbours_full[reversed_map[j]-1]) - 2 #Don't want to count the i,j edge

    if count > 0:

        for m in range(count):

            rho_node1.append((i-1)*N + j)

    for a in find_indices(i, node1_reduced):

        if node2_reduced[a] != j:

            rho_node2.append((i-1)*N + node2_reduced[a])

    for a in find_indices(i, node2_reduced):

        if node1_reduced[a] != j:

            rho_node2.append((node1_reduced[a]-1)*N + i)

    for a in find_indices(j, node1_reduced):

        if node2_reduced[a] != i:

            rho_node2.append((j-1)*N + node2_reduced[a])

    for a in find_indices(j, node2_reduced):

        if node1_reduced[a] != i:

            rho_node2.append((node1_reduced[a]-1)*N + j)


In [109]:
len(rho_node1) == len(rho_node2)

True

In [110]:
#Now add the case of the correlation p_ii

for i in range(1,N+1):

    count = int(neighbours_full[reversed_map[i]-1])
        
    if count > 0:

        for m in range(count):

            rho_node1.append((i-1)*N + i)

        for a in find_indices(i, node1_reduced):

            rho_node2.append((i-1)*N + node2_reduced[a])

        for a in find_indices(i, node2_reduced):

            rho_node2.append((node1_reduced[a]-1)*N + i)


In [111]:
len(rho_node1) == len(rho_node2)

True

In [112]:
len(rho_node1)

2222

Now I want to export these lists so I can use them in R

In [114]:
np.savetxt("pre_icar_data/node1.csv", node1_reduced, delimiter=",", fmt="%d")

np.savetxt("pre_icar_data/node2.csv", node2_reduced, delimiter=",", fmt="%d")

In [116]:
y = speeds.ws10.loc["1968-07-01 00:00:00"].unstack()

In [117]:
lst = y.to_numpy().flatten(order='F')

In [118]:
np.savetxt("pre_icar_data/y_ws10.csv", lst[df_list], delimiter=",", fmt="%.6f")

In [119]:
test = rainfall.droplevel(level=[0,1])
vals = test.set_index([test.valid_time, test.index])
vals = vals.rename_axis(index={"valid_time":"time"})

In [120]:
y2 = vals.tp.loc["1968-07-01 00:00:00"].unstack()

In [121]:
lst2 = y2.to_numpy()[::-1, :].flatten(order='F')

In [122]:
np.savetxt("pre_icar_data/y_tp.csv", lst2[df_list], delimiter=",", fmt="%.6f")

In [123]:
np.savetxt("pre_icar_data/rho_node1.csv", rho_node1, delimiter=",", fmt="%d")

np.savetxt("pre_icar_data/rho_node2.csv", rho_node2, delimiter=",", fmt="%d")